<a href="https://colab.research.google.com/github/leex5089/childcareaccess/blob/master/CMO_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by WL 12/1/2021: The purpose of this memo is to understand the issue related to the subgroup analysis conducted in 2017 CMO study, and discuss potential solution to the problem.

In [ ]:
#import Stata Module
import stata_setup 
# Stata path directory
stata_setup.config("C:/Program Files/Stata17", "mp") 

<h1>Creating Simulated Data for Illustration</h1>

In order to understand the issue and possible solutions to the models used in the 2017 CMO subgroup analysis, below block of code creates simulated data. For simplicity, only School type (1.TPS 2. CMO 3. NonCMO) and lunch (0. no lunch 1. lunch) variables are included for the illustration.

In [ ]:
%%stata
clear
# set number of observations of simulated data to have N=100,000
set obs 100000 
# designate seed number to fixate simulated results
set seed 346787 

#create error term variable that has normal distribution with mean 0 SD -1
gen u = rnormal()
#create temp variable that has normal distribution with mean 0 SD -1
gen temp = rnormal()  
#create lunch binary variable that indicates lunch or no lunch.
xtile lunch = temp, nq(2)
drop temp


#recode lunch variable

tab lunch,gen(lunch_)

rename lunch_1 lunch_no
rename lunch_2  lunch_yes

replace lunch=0 if lunch==1
replace lunch=1 if lunch==2


#check lunch variable 0 if no lunch 1 if lunch
tab lunch


#create temp variable that has normal distribution with mean 0 SD -1
gen temp = rnormal()
#create stype categorical variable that indicates 1.TPS 2.CMO 3.NonCMO.
xtile stype = temp, nq(3)
drop temp


#recode lunch variable

tab stype,gen(stype_)
 
rename stype_1 tps 
rename stype_2 cmo
rename stype_3 noncmo
 
#check stype variable
tab stype



. clear

. # set number of observations of simulated data to have N=100,000
Unknown #command
. set obs 100000 
Number of observations (_N) was 0, now 100,000.

. # designate seed number to fixate simulated results
Unknown #command
. set seed 346787 

. 
. #create error term variable that has normal distribution with mean 0 SD -1
Unknown #command
. gen u = rnormal()

. #create temp variable that has normal distribution with mean 0 SD -1
Unknown #command
. gen temp = rnormal()  

. #create lunch binary variable that indicates lunch or no lunch.
Unknown #command
. xtile lunch = temp, nq(2)

. drop temp

. 
. 
. #recode lunch variable
Unknown #command
. 
. tab lunch,gen(lunch_)

2 quantiles |
    of temp |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |     50,000       50.00       50.00
          2 |     50,000       50.00      100.00
------------+-----------------------------------
      Total |    100,000      100.00

. 
. rename lunch_

<h1>Data Generating Process</h1>


Now we have random variables generated in the above step, we generate outcome variable that has following data generating process (EQ1). There are three parts to this model, 1. Main effect of lunch, 2. Main effect of School Type, 3.Interaction effects between school type and lunch status:

$y = \overbrace{1}^{\text{Constant}}+\overbrace{ 3 \times lunch\_no+2 \times lunch\_yes}^{\text{Main Effects for Lunch}}+\overbrace{ 3 \times tps +4 \times cmo+7 \times noncmo}^{\text{Main Effects for School Type}}+\overbrace{ 5 \times lunch\_yes \times cmo -5 \times lunch\_yes \times noncmo }^{\text{Interaction Effects}}+ \overbrace{u}^{\text{Error term}}  $



In [ ]:
%%stata

#create a hypotetical model (Data Generating Process using EQ1)
#The model has both main effects and interaction effects as described above.
 
gen y = 1+3*lunch_no+2*lunch_yes+3*tps +4*cmo+7*noncmo+ 5*lunch_yes*cmo -5*lunch_yes*noncmo + u 


. 
. #create a hypotetical model (Data Generating Process using EQ1)
Unknown #command
. #The model has both main effects and interaction effects as described above.
Unknown #command
.  
. gen y = 1+3*lunch_no+2*lunch_yes+3*tps +4*cmo+7*noncmo+ 5*lunch_yes*cmo -5*lu
> nch_yes*noncmo + u 

. 


<h1>Estimation using regression models</h1>


<h4>1. Estimation model 1 (M1): Main effects only model  </h4>

In [ ]:
%%stata
#Running regression on the simulated data
#Estimation model 1: Main effects model only 

reg y i.lunch i.stype,baselevels


. #Running regression on the simulated data
Unknown #command
. #Estimation model 1: Main effects model only 
Unknown #command
. 
. reg y i.lunch i.stype,baselevels

      Source |       SS           df       MS      Number of obs   =   100,000
-------------+----------------------------------   F(3, 99996)     =  15009.08
       Model |  231742.367         3  77247.4558   Prob > F        =    0.0000
    Residual |  514651.041    99,996  5.14671628   R-squared       =    0.3105
-------------+----------------------------------   Adj R-squared   =    0.3105
       Total |  746393.408    99,999  7.46400872   Root MSE        =    2.2686

------------------------------------------------------------------------------
           y | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       lunch |
          0  |          0  (base)
          1  |  -1.010559   .0143483   -70.43   0.000    -1.038681   -.98

Based on the estimation result above, we have estimates on following parameters:
<p><li>constant</li></p><ul>  =7  ($=  \overbrace{3 \times lunch\_no+ 3 \times tps}^{\text{Reference group}}+\overbrace{1}^{\text{Constant}})  $</ul>

<p><li>lunch</li></p><ul>=-1 ($=2 \times lunch\_yes- 3 \times lunch\_no) $</ul>


<p><li>stype.2(CMO)</li></p><ul> =3.5 ($=(4 \times cmo- 3 \times tps)+(\overbrace{\frac{1}{2} \times 5 \times lunch\_yes \times cmo}^{\text{Half of CMO students receives lunch}})) $ </ul> 
 

<p><li>stype.3(NonCMO)</li></p><ul> =1.5 ($=(7 \times noncmo- 3 \times tps)-(\overbrace{\frac{1}{2} \times 5 \times lunch\_yes \times noncmo}^{\text{Half of NonCMO students receives lunch}})) $ </ul> 

As a reminder, EQ1 (true simulated model) is:

$y = \overbrace{1}^{\text{Constant}}+\overbrace{ 3 \times lunch\_no+2 \times lunch\_yes}^{\text{Main Effects for Lunch}}+\overbrace{ 3 \times tps +4 \times cmo+7 \times noncmo}^{\text{Main Effects for School Type}}+\overbrace{ 5 \times lunch\_yes \times cmo -5 \times lunch\_yes \times noncmo }^{\text{Interaction Effects}}+ \overbrace{u}^{\text{Error term}}  $

 <h4>2. Estimation model 2 (M2): Interaction effects only model  </h4>

In [ ]:
%%stata
#Estimation model 2: Interaction effects model only 

reg y i.lunch#i.stype ,baselevels



. #Estimation model 2: Interaction effects model only 
Unknown #command
. 
. reg y i.lunch#i.stype ,baselevels

      Source |       SS           df       MS      Number of obs   =   100,000
-------------+----------------------------------   F(5, 99994)     >  99999.00
       Model |  647102.231         5  129420.446   Prob > F        =    0.0000
    Residual |  99291.1767    99,994  .992971346   R-squared       =    0.8670
-------------+----------------------------------   Adj R-squared   =    0.8670
       Total |  746393.408    99,999  7.46400872   Root MSE        =    .99648

------------------------------------------------------------------------------
           y | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
 lunch#stype |
        0 1  |          0  (base)
        0 2  |   1.020405   .0108988    93.63   0.000     .9990434    1.041766
        0 3  |   4.007916   .0109215   366.97  

Based on the estimation result above, we have estimates on following parameters:
<p><li>Constant</li></p><ul>  =7  ($=  \overbrace{3 \times lunch\_no+ 3 \times tps}^{\text{Reference group}}+\overbrace{1}^{\text{Constant}})  $</ul>

<p><li>$lunch\_no \times cmo$</li></p><ul>=1 ($=4 \times cmo- 3 \times tps) $</ul>
<p><li>$lunch\_no \times noncmo$</li></p><ul>=4 ($=7 \times noncmo- 3 \times tps) $</ul>
<p><li>$lunch\_yes \times tps$</li></p><ul>=-1 ($=2 \times lunch\_yes- 3 \times lunch\_no) $</ul>
<p><li>$lunch\_yes \times cmo$</li></p><ul>=5 ($=(2 \times lunch\_yes- 3 \times lunch\_no)+(4 \times cmo- 3 \times tps)+(5 \times lunch\_yes \times cmo)) $</ul>
<p><li>$lunch\_yes \times noncmo$</li></p><ul>=-2 ($=(2 \times lunch\_yes- 3 \times lunch\_no)+(7 \times noncmo- 3 \times tps)-(5 \times lunch\_yes \times noncmo)) $</ul>

 

As a reminder, EQ1 (true simulated model) is:

$y = \overbrace{1}^{\text{Constant}}+\overbrace{ 3 \times lunch\_no+2 \times lunch\_yes}^{\text{Main Effects for Lunch}}+\overbrace{ 3 \times tps +4 \times cmo+7 \times noncmo}^{\text{Main Effects for School Type}}+\overbrace{ 5 \times lunch\_yes \times cmo -5 \times lunch\_yes \times noncmo }^{\text{Interaction Effects}}+ \overbrace{u}^{\text{Error term}}  $

 <h4>3. Estimation model 3 (M3): Full model (both main and interaction effects) </h4>

In [ ]:
%%stata
#Estimation model 3: Full model (both main and interaction effects)


reg y i.lunch##i.stype,baselevels



. #Estimation model 3: Full model (both main and interaction effects)
Unknown #command
. 
. 
. reg y i.lunch##i.stype,baselevels

      Source |       SS           df       MS      Number of obs   =   100,000
-------------+----------------------------------   F(5, 99994)     >  99999.00
       Model |  647102.231         5  129420.446   Prob > F        =    0.0000
    Residual |  99291.1767    99,994  .992971346   R-squared       =    0.8670
-------------+----------------------------------   Adj R-squared   =    0.8670
       Total |  746393.408    99,999  7.46400872   Root MSE        =    .99648

------------------------------------------------------------------------------
           y | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       lunch |
          0  |          0  (base)
          1  |  -1.001947   .0109159   -91.79   0.000    -1.023342   -.9805518
             |
       stype |

Based on the estimation result above, we have estimates on following parameters:
<p><li>Constant</li></p><ul>  =7  ($=  \overbrace{3 \times lunch\_no+ 3 \times tps}^{\text{Reference group}}+\overbrace{1}^{\text{Constant}})  $</ul>

<p><li>lunch</li></p><ul>=-1 ($=2 \times lunch\_yes- 3 \times lunch\_no) $</ul>


<p><li>stype.2(CMO)</li></p><ul> =1 ($=4 \times cmo- 3 \times tps) $ </ul> 
 

<p><li>stype.3(NonCMO)</li></p><ul> =4 ($=7 \times noncmo- 3 \times tps) $ </ul> 

<p><li>$lunch\_yes \times cmo$</li></p><ul>=5 ($= 5 \times lunch\_yes \times cmo) $</ul>
<p><li>$lunch\_yes \times noncmo$</li></p><ul>=-5 ($= -5 \times lunch\_yes \times cmo) $</ul>
 

As a reminder, EQ1 (true simulated model) is:

$y = \overbrace{1}^{\text{Constant}}+\overbrace{ 3 \times lunch\_no+2 \times lunch\_yes}^{\text{Main Effects for Lunch}}+\overbrace{ 3 \times tps +4 \times cmo+7 \times noncmo}^{\text{Main Effects for School Type}}+\overbrace{ 5 \times lunch\_yes \times cmo -5 \times lunch\_yes \times noncmo }^{\text{Interaction Effects}}+ \overbrace{u}^{\text{Error term}}  $

<h1>Discussing the issues in the previous subgroup analysis</h1>


1.Based on the simulated data and estimation results, we can infer that addition of the coefficient on CMO in the main effect only model (M1) to the coefficient on lunch in the interaction only model (M2) tend to bias the true effect of the $lunch \times cmo$ on $Y$ (i.e., $5+3.5 > 5$)


2.Based on the simulated data and estimation results, we can infer that addition of the coefficient on NonCMO in the main effect only model (M1) to the coefficient on lunch in the interaction only model (M2) tend to bias the true effect of the $lunch \times noncmo$ on $Y$ (i.e., $-2+1.5 >-5$)

Note: The same addition of coefficients from (M1) and (M2) was conducted for ELL and SPED in the 2017 CMO analysis, but not for race/ethnicity breakout. 


<h1>Potional solution to the problem</h1>


1. The use of full model (M3) that includes both main and interaction effects in the model recovers the unbiased estimated results for both coefficients of the interaction terms ($\overbrace{ 5 \times lunch\_yes \times cmo -5 \times lunch\_yes \times noncmo }^{\text{Interaction Effects}}$). The interpretation would be that "The CMO lunch students show 5 units higher of outcome variable than the no-lunch CMO students, controlling for other factors included in the model".  




2. The use of interaction only model (Model 2) can also recovers the unbiased estimated results, but it aggregates the main effects of school type and lunch to the interaction effects between school type and lunch. One thing to keep in mind when choosing the interaction only model is that as the number of variables increases, the number of interaction terms increase exponentially (eg., when there is 5 binary variables, we need $2^{5}$=32 terms in the model). For example, a Data generating process with 5 binary variables is assumed below. The results shows that the coefficients and interpretation should in reference to the base group defined at v1=1, v2=1, v3=1, v4=1, v5=1.




In [ ]:
%%stata

 
 ***
 
 clear 

set obs 100000
set seed 346787
gen u = rnormal()
gen temp = rnormal()

xtile v1 = temp, nq(2)
drop temp
gen temp = rnormal()
xtile v2 = temp, nq(2)
drop temp
gen temp = rnormal()
xtile v3 = temp, nq(2)
drop temp
gen temp = rnormal()
xtile v4 = temp, nq(2)
drop temp
gen temp = rnormal()
xtile v5 = temp, nq(2)
drop temp
 
 
 
gen y = 1+3*v1+2*v2+3*v3 +4*v4+7*v5+ u 

 
reg y i.v1#i.v2#i.v3#i.v4#i.v5,baselevels
 


. 
.  
.  ***
.  
.  clear 

. 
. set obs 100000
Number of observations (_N) was 0, now 100,000.

. set seed 346787

. gen u = rnormal()

. gen temp = rnormal()

. 
. xtile v1 = temp, nq(2)

. drop temp

. gen temp = rnormal()

. xtile v2 = temp, nq(2)

. drop temp

. gen temp = rnormal()

. xtile v3 = temp, nq(2)

. drop temp

. gen temp = rnormal()

. xtile v4 = temp, nq(2)

. drop temp

. gen temp = rnormal()

. xtile v5 = temp, nq(2)

. drop temp

.  
.  
.  
. gen y = 1+3*v1+2*v2+3*v3 +4*v4+7*v5+ u 

. 
.  
. reg y i.v1#i.v2#i.v3#i.v4#i.v5,baselevels

      Source |       SS           df       MS      Number of obs   =   100,000
-------------+----------------------------------   F(31, 99968)    =  70862.64
       Model |  2181380.58        31  70367.1155   Prob > F        =    0.0000
    Residual |  99268.9514    99,968  .993007276   R-squared       =    0.9565
-------------+----------------------------------   Adj R-squared   =    0.9565
       Total |  2280649.53    99,999  22.